Creating Fact and Dimension Tables with Star Schema

Walk through the basics of modeling data using Fact and Dimension tables. We will create both Fact and Dimension tables and show how this is a basic element of the Star Schema.

This exercise will be more challenging than the last. Use the information provided to create the tables and write the insert statements.

In [12]:
#Import the library
import psycopg2

In [13]:
#Create a connection to the database

try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=myfirstdb user=postgres password=Lanremj06")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

#Next use that connect to get a cursor that we will use to execute queries.
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

We will use automactic commit so that each action is commited without having to call conn.commit() after each command. The ability to rollback and commit transactions is a feature of Relational Databases.

In [14]:
conn.set_session(autocommit=True)

Imagine We work at an online Music Store. There will be many tables in our database, but let's just focus on 4 tables around customer purchases.

From these 4 tables we can make a "STAR" schema. We will have one fact table (the center of the star) and 3 dimension tables that are coming from it.

In [15]:
#Create the Fact table and insert the data into the table

try:
    cur.execute('''CREATE TABLE customer_transactions (customer_id int, store_id int, amount_spent float)''')
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

#Insert into all tables
try:
    cur.execute('''INSERT INTO customer_transactions (customer_id, store_id, amount_spent) \
                VALUES (%s, %s, %s)''', (1, 1, 20.5))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

try:
    cur.execute('''INSERT INTO customer_transactions (customer_id, store_id, amount_spent) \
                VALUES (%s, %s, %s)''', (2, 1, 35.21))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

In [16]:
#Create the Dimension tables and insert data into those tables
#Customer table

try:
    cur.execute('''
        CREATE TABLE customer
        (customer_id int, name varchar, rewards varchar)
        ''')
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)


try:
    cur.execute('''
        INSERT INTO customer
            (customer_id, name, rewards)
            VALUES (%s, %s, %s)''', (1, "Amanda", "Y"))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

try:
    cur.execute('''
        INSERT INTO customer
            (customer_id, name, rewards)
            VALUES (%s, %s, %s)''', (2, "Toby", "N"))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

In [17]:
#Items_purchased table

try:
    cur.execute('''
        CREATE TABLE items_purchased
            (customer_id int, item_number int, item_name varchar)
            ''')
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute('''
        INSERT INTO items_purchased
            (customer_id, item_number, item_name)
            VALUES (%s, %s, %s)
            ''', (1, 1, 'Rubber Soul'))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

try:
    cur.execute('''
        INSERT INTO items_purchased
            (customer_id, item_number, item_name)
            VALUES (%s, %s, %s)
            ''', (2, 3, 'Let It Be'))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

In [18]:
#Store table

try:
    cur.execute('''
        CREATE TABLE store (store_id int, state varchar)
        ''')
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute('''
        INSERT INTO store (store_id, state)
            VALUES (%s, %s)
            ''', (1, "CA"))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

try:
    cur.execute('''
        INSERT INTO store (store_id, state)
            VALUES (%s, %s)
            ''', (2, "WA"))
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print(e)

Now we run the following queries on this data easily because of utilizing the Fact/ Dimension and Star Schema:


Query 1: Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.

Query 2: How much did Customer 2 spend?

In [19]:
#Query 1:

try:
    cur.execute('''
        SELECT customer.name, store.store_id, store.state, items_purchased.item_name, customer.rewards
        FROM customer_transactions
        JOIN customer ON customer_transactions.customer_id = customer.customer_id
        JOIN store ON customer_transactions.store_id = store.store_id
        JOIN items_purchased ON customer_transactions.customer_id = items_purchased.customer_id
        WHERE customer_transactions.amount_spent > 30
        ''')
except psycopg2.Error as e:
    print("Error: select *")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Toby', 1, 'CA', 'Let It Be', 'N')


In [20]:
#Query 2:

try:
    cur.execute('''
        SELECT customer_id, SUM(amount_spent)
        FROM customer_transactions
        WHERE customer_id = 2
        GROUP BY customer_id
        ''')
except psycopg2.Error as e:
    print("Error: select*")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(2, 35.21)


Summary

We can see here from this elegant schema that we were:
1) able to get "facts/metrics" from our fact table (how much each store sold)
2) information about our customers that will allow us to do more indepth analytics to get answers to business questions by utilizing our fact and dimension tables.

In [21]:
#Drop the tables
try: 
    cur.execute("DROP TABLE customer_transactions")
    cur.execute("DROP TABLE customer")
    cur.execute("DROP TABLE store")
    cur.execute("DROP TABLE items_purchased")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

In [22]:
#Close cursor and connection
cur.close()
conn.close()